In [1]:
import os
import sys
import pandas as pd
import pyodbc
from datetime import datetime
from dateutil import parser
from glob import glob


In [2]:
os.chdir(r"N:\07_Geology\07.9_QAQC\40_Data\43_Assays\ALSChemex\02_Data")
input_files = glob("*.csv")

In [3]:
conn = pyodbc.connect(r'DSN=OKLG_QAQC;Trusted_Connection=True')
if conn:
    print "connection created"
for f in input_files:
    testsql = "Select * from LabJobCert where CertNum = '{}'".format(f.replace('.csv',''))
    test = conn.execute(testsql)
    if not test.fetchone():
        print "Inserting cert {}".format(f)
        input_file = r"N:\07_Geology\07.9_QAQC\40_Data\43_Assays\ALSChemex\02_Data\{}".format(f)
        csv = pd.read_csv(input_file, sep = ',', skiprows = [0,1,2,3,4,5,6,7,11,12,13,14,15,16], header = [0,1,2], index_col = 0)
        header = pd.read_csv(input_file, nrows = 8, index_col = 0, usecols= [0,1])
        print header
        row = [datetime.strftime(datetime.today(), "%Y%m%d %H:%M:%S %p"), header.index.name, header.ix['CLIENT'][0], \
               int(header.ix['# of SAMPLES'][0]),datetime.strftime(parser.parse(header.ix['DATE RECEIVED'][0], \
               dayfirst = True), "%Y%m%d %H:%M:%S %p"), \
               datetime.strftime(parser.parse(header.ix['DATE COMPLETED'][0], dayfirst = True), "%Y%m%d %H:%M:%S %p"), \
               header.ix['PROJECT'][0],"NULL",header.ix['PO NUMBER'][0],"ALSChemex",header.index.name]
        conn.execute("INSERT INTO dbo.LabJobCert (LoadDate, CertNum, Client, NumSamples,\
                 DateRecieved, DateFinalised, Project, Comment, PONumber, Lab, OrigFileName) \
                 values ('{}','{}','{}','{}','{}','{}','{}',{},'{}','{}','{}')".format(*row))
        conn.commit()
        for this in csv.iterrows():
            for x, y in  this[1].iteritems():
                if x[2] <> 'NA' and not pd.isnull(y):
                    if this[0].startswith("Blank"):
                        SampleType = 'Lab Blank'
                        ref = 'Blank'
                    elif this[0].startswith('Ch:'):
                        SampleType = 'Lab Check'
                        ref = this[0].replace('Ch:','').strip()
                    elif this[0].startswith('St'):
                        SampleType = 'Lab Standard'
                        ref = this[0][this[0].index(":")+1: this[0].index("_")]
                    else:
                        SampleType = 'Original'
                        ref = None
                    name = "{1}_{0}_{2}".format(*x).replace(" ","")    
                    row =  [datetime.strftime(datetime.today(), "%Y%m%d %H:%M:%S %p"), header.index.name, this[0], SampleType, name, ref, y, header.index.name, 1, x[1], x[2], x[0]]
                    conn.execute("INSERT INTO dbo.LabJobResults (LoadDate, CertNum, SampleID, SampleType, Name, Reference, Value, OrigFileName, Preferred, Element, Unit, Method)\
                    values ('{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}')".format(*row))
                    conn.commit()


connection created
Inserting cert TM17208256.csv
                                                   Finalized
TM17208256                                                  
CLIENT                OSIKLI - Canadian Malartic Corporation
# of SAMPLES                                              66
DATE RECEIVED                                         270917
DATE COMPLETED                                        011017
PROJECT                                                   UB
CERTIFICATE COMMENTS                                     NaN
PO NUMBER                                             KLS-47
NaN                                                  Cu-AA45


In [4]:
conn.close()